In [1]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the fourth GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
        
tf.compat.v1.disable_v2_behavior()

1 Physical GPUs, 1 Logical GPUs
Instructions for updating:
non-resource variables are not supported in the long term


In [23]:
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D, Add, Multiply, GlobalAveragePooling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
import keras.backend as K
import numpy as np
from PIL import Image
import os
import matplotlib.pyplot as plt

In [68]:
# Preview image Frame
PREVIEW_ROWS = 4
PREVIEW_COLS = 7
PREVIEW_MARGIN = 4
SAVE_FREQ = 100

# Size vector to generate images from
NOISE_SIZE = 100

# Configuration
EPOCHS = 20000 # number of iterations
BATCH_SIZE = 8
GENERATE_RES = 3
IMAGE_SIZE = 128 # rows/cols
IMAGE_CHANNELS = 3

In [4]:
training_data = np.load('cubism_data.npy')

In [46]:
def build_discriminator(image_shape):
    Inputs = Input(image_shape)
    
    X = Conv2D(filters = 32, kernel_size=(3,3), strides=(2,2), padding='same')(Inputs)
    
    X_shortcut = X
    
    X = LeakyReLU(alpha=0.2)(X)
    X = Dropout(0.25)(X)
    X = Conv2D(filters = 64, kernel_size=(3,3), strides=(2,2), padding='same')(X)
    X = ZeroPadding2D(padding=((0,1),(0,1)))(X)
    X = BatchNormalization(momentum=0.8)(X)
    
    X_shortcut = Conv2D(filters = 64, kernel_size=(1,1), strides=(2,2))(X_shortcut)
    X_shortcut = ZeroPadding2D(padding=((0,1),(1,0)))(X_shortcut)
    X_shortcut = LeakyReLU(alpha=0.2)(X_shortcut)
    
    X = Add()([X, X_shortcut])
    
    X_shortcut = X
    
    X = LeakyReLU(alpha=0.2)(X)
    X = Dropout(0.25)(X)
    X = Conv2D(filters = 128, kernel_size=(3,3), strides=(2,2), padding='same')(X)
    X = BatchNormalization(momentum=0.8)(X)
    
    X = LeakyReLU(alpha=0.2)(X)
    X = Dropout(0.25)(X)
    X = Conv2D(filters = 256, kernel_size=(3,3), strides=(1,1), padding='same')(X)
    X = BatchNormalization(momentum=0.8)(X)
    
    X_shortcut = Conv2D(filters = 256, kernel_size=(1,1), strides=(2,2))(X_shortcut)
    X_shortcut = LeakyReLU(alpha=0.2)(X_shortcut)
    
    X = Add()([X, X_shortcut])
    
    X = LeakyReLU(alpha=0.2)(X)
    X = Dropout(0.25)(X)
    X = Conv2D(filters = 512, kernel_size=(3,3), strides=(1,1), padding='valid')(X)
    X = BatchNormalization(momentum=0.8)(X)    
    
    X = LeakyReLU(alpha=0.2)(X)
    X = Dropout(0.25)(X)
    X = Flatten()(X)
    
    X = Dense(1, activation = 'sigmoid')(X)
    
    return Model(inputs = Inputs, outputs=X)
    
    '''model = Sequential()
    model.add(Conv2D(32, kernel_size=3, strides=2,
    input_shape=image_shape, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(512, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    input_image = Input(shape=image_shape)
    validity = model(input_image)
    return Model(input_image, validity)'''

In [47]:
test = build_discriminator((128,128,3))
test.summary()

Model: "model_11"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_20 (InputLayer)           (None, 128, 128, 3)  0                                            
__________________________________________________________________________________________________
conv2d_98 (Conv2D)              (None, 64, 64, 32)   896         input_20[0][0]                   
__________________________________________________________________________________________________
leaky_re_lu_83 (LeakyReLU)      (None, 64, 64, 32)   0           conv2d_98[0][0]                  
__________________________________________________________________________________________________
dropout_61 (Dropout)            (None, 64, 64, 32)   0           leaky_re_lu_83[0][0]             
___________________________________________________________________________________________

In [63]:
def build_generator(noise_size, channels):
    Inputs = Input((noise_size,))
    
    X = Dense(4*4*256, activation='relu')(Inputs)
    X = Reshape((4,4,256))(X)
    
    X_shortcut = X
    
    X = UpSampling2D()(X)
    X = Conv2D(filters = 256, kernel_size=(3,3), padding='same')(X)
    X = BatchNormalization(momentum=0.8)(X)
    X = Activation('relu')(X)
    
    X_shortcut = GlobalAveragePooling2D()(X_shortcut)
    X_shortcut = Reshape((1,1,256))(X_shortcut)
    X_shortcut = Conv2D(filters = 64, kernel_size = (1,1))(X_shortcut)
    X_shortcut = Conv2D(filters = 256, kernel_size = (1,1))(X_shortcut)
    X_shortcut = Activation('relu')(X_shortcut)
    
    X = Multiply()([X, X_shortcut])
    
    X = UpSampling2D()(X)
    X = Conv2D(filters = 256, kernel_size=(3,3), padding='same')(X)
    X = BatchNormalization(momentum=0.8)(X)
    X = Activation('relu')(X)
    
    X_shortcut = X
    
    for i in range(GENERATE_RES):
        X = UpSampling2D()(X)
        X = Conv2D(filters = 256, kernel_size=(3,3), padding='same')(X)
        X = BatchNormalization(momentum=0.8)(X)
        X = Activation('relu')(X)
    
    X_shortcut = GlobalAveragePooling2D()(X_shortcut)
    X_shortcut = Reshape((1,1,256))(X_shortcut)
    X_shortcut = Conv2D(filters = 64, kernel_size = (1,1))(X_shortcut)
    X_shortcut = Conv2D(filters = 256, kernel_size = (1,1))(X_shortcut)
    X_shortcut = Activation('relu')(X_shortcut)
    
    X = Multiply()([X, X_shortcut])
    
    X = Conv2D(channels, kernel_size=(3,3), padding='same')(X)
    X = Activation('tanh')(X)
    
    return Model(inputs = Inputs, outputs = X)
    '''model = Sequential()
    model.add(Dense(4 * 4 * 256, activation='relu',       input_dim=noise_size))
    model.add(Reshape((4, 4, 256)))
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    for i in range(GENERATE_RES):
        model.add(UpSampling2D())
        model.add(Conv2D(256, kernel_size=3, padding='same'))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation('relu'))
    model.summary()
    model.add(Conv2D(channels, kernel_size=3, padding='same'))
    model.add(Activation('tanh'))
    inputs = Input(shape=(noise_size,))
    generated_image = model(inputs)
    
    return Model(inputs, generated_image)'''

In [64]:
test = build_generator(128, 3)

test.summary()

Model: "model_19"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_30 (InputLayer)           (None, 128)          0                                            
__________________________________________________________________________________________________
dense_20 (Dense)                (None, 4096)         528384      input_30[0][0]                   
__________________________________________________________________________________________________
reshape_20 (Reshape)            (None, 4, 4, 256)    0           dense_20[0][0]                   
__________________________________________________________________________________________________
global_average_pooling2d_12 (Gl (None, 256)          0           reshape_20[0][0]                 
___________________________________________________________________________________________

In [65]:
def save_images(cnt, noise):
    image_array = np.full((
        PREVIEW_MARGIN + (PREVIEW_ROWS * (IMAGE_SIZE + PREVIEW_MARGIN)),
        PREVIEW_MARGIN + (PREVIEW_COLS * (IMAGE_SIZE + PREVIEW_MARGIN)), 3),
        255, dtype=np.uint8)
    
    generated_images = generator.predict(noise)
    
    generated_images = 0.5 * generated_images + 0.5
    image_count = 0
    for row in range(PREVIEW_ROWS):
        for col in range(PREVIEW_COLS):
            r = row * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            c = col * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
            image_array[r:r + IMAGE_SIZE, c:c +
                        IMAGE_SIZE] = generated_images[image_count] * 255
            image_count += 1
    output_path = 'output'
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    filename = os.path.join(output_path, f"trained-{cnt}.png")
    im = Image.fromarray(image_array)
    im.save(filename)

In [66]:
image_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)

optimizer = Adam(lr=1.5e-4, beta_1=0.5)

discriminator = build_discriminator(image_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

generator = build_generator(NOISE_SIZE, IMAGE_CHANNELS)

random_input = Input(shape=(NOISE_SIZE,))

generated_image = generator(random_input)

discriminator.trainable = False

validity = discriminator(generated_image)

combined = Model(random_input, validity)
combined.compile(loss = 'binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [69]:
y_real = np.ones((BATCH_SIZE, 1))
y_fake = np.zeros((BATCH_SIZE, 1))

fixed_noise = np.random.normal(0,1, (PREVIEW_ROWS* PREVIEW_COLS, NOISE_SIZE))

cnt = 1

for epoch in range(EPOCHS):
    idx = np.random.randint(0, training_data.shape[0], BATCH_SIZE)
    x_real = training_data[idx]
    
    noise = np.random.normal(0,1, (BATCH_SIZE, NOISE_SIZE))
    x_fake = generator.predict(noise)
    
    discriminator_metric_real = discriminator.train_on_batch(x_real, y_real)
    
    discriminator_metric_generated = discriminator.train_on_batch(x_fake, y_fake)
    
    discriminator_metric = 0.5 * np.add(discriminator_metric_real, discriminator_metric_generated)
    
    generator_metric = combined.train_on_batch(noise, y_real)
    
    if(epoch % SAVE_FREQ == 0):
        save_images(cnt, fixed_noise)
        cnt += 1
        print(f"{epoch} epoch, Discriminator accuracy: {100*discriminator_metric[1]}, Generator accuracy: {100*generator_metric[1]}")

0 epoch, Discriminator accuracy: 59.375, Generator accuracy: 0.0
100 epoch, Discriminator accuracy: 50.0, Generator accuracy: 37.5
200 epoch, Discriminator accuracy: 31.25, Generator accuracy: 12.5
300 epoch, Discriminator accuracy: 81.25, Generator accuracy: 0.0
400 epoch, Discriminator accuracy: 50.0, Generator accuracy: 37.5
500 epoch, Discriminator accuracy: 81.25, Generator accuracy: 25.0
600 epoch, Discriminator accuracy: 25.0, Generator accuracy: 37.5
700 epoch, Discriminator accuracy: 43.75, Generator accuracy: 0.0
800 epoch, Discriminator accuracy: 81.25, Generator accuracy: 12.5
900 epoch, Discriminator accuracy: 68.75, Generator accuracy: 12.5
1000 epoch, Discriminator accuracy: 93.75, Generator accuracy: 25.0
1100 epoch, Discriminator accuracy: 50.0, Generator accuracy: 75.0
1200 epoch, Discriminator accuracy: 93.75, Generator accuracy: 25.0
1300 epoch, Discriminator accuracy: 37.5, Generator accuracy: 37.5
1400 epoch, Discriminator accuracy: 62.5, Generator accuracy: 25.0


12200 epoch, Discriminator accuracy: 81.25, Generator accuracy: 12.5
12300 epoch, Discriminator accuracy: 68.75, Generator accuracy: 0.0
12400 epoch, Discriminator accuracy: 93.75, Generator accuracy: 0.0
12500 epoch, Discriminator accuracy: 87.5, Generator accuracy: 12.5
12600 epoch, Discriminator accuracy: 81.25, Generator accuracy: 0.0
12700 epoch, Discriminator accuracy: 93.75, Generator accuracy: 0.0
12800 epoch, Discriminator accuracy: 100.0, Generator accuracy: 12.5
12900 epoch, Discriminator accuracy: 93.75, Generator accuracy: 0.0
13000 epoch, Discriminator accuracy: 81.25, Generator accuracy: 0.0
13100 epoch, Discriminator accuracy: 87.5, Generator accuracy: 0.0
13200 epoch, Discriminator accuracy: 81.25, Generator accuracy: 0.0
13300 epoch, Discriminator accuracy: 75.0, Generator accuracy: 0.0
13400 epoch, Discriminator accuracy: 87.5, Generator accuracy: 0.0
13500 epoch, Discriminator accuracy: 93.75, Generator accuracy: 12.5
13600 epoch, Discriminator accuracy: 81.25, Gene